In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
!pip install lazypredict
from lazypredict.Supervised import LazyClassifier

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 27.7 MB/s eta 0:00:00a 0:00:01


In [4]:
df=pd.read_csv('../Resources/watson_healthcare_modified.csv')

In [5]:
##Version 2: Remove additional columns
df = df.drop(columns = ['EmployeeID', 'EmployeeCount', 'StandardHours', 'TrainingTimesLastYear', 'MonthlyRate', 'DailyRate', 'HourlyRate', 'Over18'])

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1676 entries, 0 to 1675
Data columns (total 27 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1676 non-null   int64 
 1   Attrition                 1676 non-null   object
 2   BusinessTravel            1676 non-null   object
 3   Department                1676 non-null   object
 4   DistanceFromHome          1676 non-null   int64 
 5   Education                 1676 non-null   int64 
 6   EducationField            1676 non-null   object
 7   EnvironmentSatisfaction   1676 non-null   int64 
 8   Gender                    1676 non-null   object
 9   JobInvolvement            1676 non-null   int64 
 10  JobLevel                  1676 non-null   int64 
 11  JobRole                   1676 non-null   object
 12  JobSatisfaction           1676 non-null   int64 
 13  MaritalStatus             1676 non-null   object
 14  MonthlyIncome           

In [6]:
target='Attrition'
# remove target
X=df.drop(columns=['Attrition']).copy()
# get list of columns
cols=list(X.columns)
# start choices dictionary to capture categorical options for HTML
choices={col: None for col in cols}

# get all categorical columns
cat_cols=X.dtypes[X.dtypes=='object'].index
# get all continuous columns
cont_cols=X.dtypes[X.dtypes!='object'].index
# get all categorical features
unique_vals_count=df[cat_cols].nunique().sum()
print(f'There should be {len(cont_cols)-len(cat_cols)+unique_vals_count} columns')

There should be 36 columns


In [7]:
# set X as only continuous features
X=X.drop(columns=cat_cols).copy()

# OHE transform categorical features
ohe=OneHotEncoder(sparse_output=False)
cat_cols_transformed=ohe.fit_transform(df[cat_cols])
categories=ohe.categories_
cat_feature_names=ohe.get_feature_names_out()

# map options to each categorical variable for HTML
for k, v in zip(cat_cols, categories): 
    choices[k]=list(v)

In [8]:
X[cat_feature_names]=cat_cols_transformed
y=df[target].copy()

In [9]:
X_train, X_test, y_train, y_test=train_test_split(X, y)

In [10]:
scaler=StandardScaler()
scaler.fit(X_train)
X_train_scaled=scaler.transform(X_train)
X_test_scaled=scaler.transform(X_test)

In [11]:
rfc=RandomForestClassifier()
rfc.fit(X_train_scaled, y_train)
print(f'Train score: {rfc.score(X_train_scaled, y_train)}')
print(f'Test score: {rfc.score(X_test_scaled, y_test)}')

Train score: 0.9968178202068417
Test score: 0.9212410501193318


In [12]:
print(classification_report(y_test, rfc.predict(X_test_scaled)))

              precision    recall  f1-score   support

          No       0.93      0.99      0.96       369
         Yes       0.84      0.42      0.56        50

    accuracy                           0.92       419
   macro avg       0.88      0.70      0.76       419
weighted avg       0.92      0.92      0.91       419



In [14]:
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)


 86%|█████████████████████████████████████      | 25/29 [00:01<00:00, 24.32it/s]

[LightGBM] [Info] Number of positive: 149, number of negative: 1108
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.018330 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 596
[LightGBM] [Info] Number of data points in the train set: 1257, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.118536 -> initscore=-2.006366
[LightGBM] [Info] Start training from score -2.006366
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


100%|███████████████████████████████████████████| 29/29 [00:01<00:00, 19.56it/s]


In [17]:
import pickle

with open('choices.pkl', 'wb') as f: 
    pickle.dump(choices, f)
        
with open('scaler.pkl', 'wb') as f: 
    pickle.dump(scaler, f)
    
with open('ohe.pkl', 'wb') as f: 
    pickle.dump(ohe, f)    
    
with open('model.pkl', 'wb') as f: 
    pickle.dump(rfc, f)